In [3]:
import networkx as nx
import sqlite3
import pandas as pd
import numpy as np

class ProvenanceExplorer:    
    def __init__(self,dbfile):
        self.dbfile = dbfile
        #self.conn = sqlite3.connect(self.dbfile)
        #self.cursor = self.conn.cursor()
        self.open_connection()

    def open_connection(self):
        self.conn = sqlite3.connect(self.dbfile)
        self.cursor = self.conn.cursor()
    
    def close_connection(self):
        self.conn.close()

    def get_number_of_state(self):
        return pd.read_sql_query("""
        SELECT max(state_id) as num_state FROM state 
        """, self.conn)

    def get_changes_each_state(self,state):
        if type(state) in (range,list):
            state=list(state)
            state = ",".join([str(x) for x in state ])
        return pd.read_sql_query("""
        SELECT (state_id-(select max(state_id)+1 from state s))*-1 as state,substr(command,33) as operation,col_id,count(1) as cell_changes FROM state 
        NATURAL JOIN content NATURAL JOIN value  NATURAL JOIN cell NATURAL JOIN state_command
        where state in ({state})
        group by state,col_id
        order by state asc        
        """.format(state=state), self.conn)

    def get_column_at_state(self,state):
        """
        col_at_state = list(self.cursor.execute("select * from col_each_state where state={}".format(state)))
        names = list(map(lambda x: x[0], self.cursor.description))   
        table_dict = {}
        for x in names:
            table_dict[x] = []
        for x in col_at_state:
            for i,y in enumerate(x):
                table_dict[names[i]].append(y)
        
        return pd.DataFrame(table_dict)
        """
        if type(state) in (range,list):
            state=list(state)
            state = ",".join([str(x) for x in state ])        
        
        #return pd.read_sql_query("select * from col_each_state where state={}".format(state), self.conn)
        return pd.read_sql_query("select * from col_each_state where state in ({state}) order by state".format(state=state), self.conn)

    def get_col_at_state_order(self,state):
        if type(state) in (range,list):
            state=list(state)
            state = ",".join([str(x) for x in state ])        

        return pd.read_sql_query("""
        SELECT * FROM (WITH RECURSIVE
        col_state_order(state,col_id,col_name,prev_col_id,level) AS (
        select state,col_id,col_name,prev_col_id,0 from col_each_state 
        where prev_col_id=-1
        UNION ALL
        SELECT a.state,a.col_id, a.col_name, a.prev_col_id,b.level+1 
        FROM col_each_state a, col_state_order b
        WHERE a.prev_col_id=b.col_id and a.state=b.state)
        SELECT state,col_id,col_name,prev_col_id,level from col_state_order
        ) where state in ({state})
        order by state,level
        """.format(state=state)
        , self.conn)

    def get_row_at_state(self,state):
        return pd.read_sql_query("select * from row_at_state where state={}".format(state), self.conn)

    def get_row_at_state(self,state):
        return pd.read_sql_query("select * from row_at_state where state={}".format(state), self.conn)

    def get_row_at_state_order(self,state):
        '''
        return pd.read_sql_query("""
        SELECT * FROM (WITH RECURSIVE
        row_state_order(state,row_id,prev_row_id,level) AS (
        select state,row_id,prev_row_id,0 from row_at_state 
        where prev_row_id=-1
        and state={}
        UNION ALL
        SELECT a.state,a.row_id,a.prev_row_id,b.level+1 
        FROM row_at_state a, row_state_order b
        WHERE a.prev_row_id=b.row_id and a.state=b.state)
        SELECT state,row_id,prev_row_id,level from row_state_order
        order by state asc)
        """.format(state)
        , self.conn)
        '''
        return pd.read_sql_query("""
        select distinct * from (
        WITH RECURSIVE
        row_state_order(state,state_id,row_id,prev_row_id,level) AS (
        select ({state}-(select max(state_id) from state s))*-1 as state
        ,{state} as state_id,rp.row_id,rp.prev_row_id,0 from row_position rp 
        where rp.state_id<={state} and
        rp.row_pos_id not IN 
        (
        select prev_row_pos_id from row_position rp2 
        where rp2.state_id<={state}
        and prev_row_pos_id>-1
        )
        and prev_row_id=-1
        UNION ALL
        SELECT a.state,a.state_id,a.row_id,a.prev_row_id,b.level+1 
        FROM (select ({state}-(select max(state_id) from state s))*-1 as state
        ,{state} as state_id,rp.row_pos_id,rp.row_id,rp.prev_row_id,rp.prev_row_pos_id from row_position rp 
        where rp.state_id<={state} and
        rp.row_pos_id not IN 
        (
        select prev_row_pos_id from row_position rp2 
        where rp2.state_id<={state}
        and prev_row_pos_id>-1
        )) a, row_state_order b
        WHERE a.prev_row_id=b.row_id and a.state_id=b.state_id)
        SELECT state,state_id,row_id,prev_row_id,level from row_state_order
        )        
        """.format(state=state),self.conn)

    def get_row_logic_to_idx(self,state,row):
        row_logic = self.get_row_at_state_order(state)
        return row_logic.iloc[row].row_id

    def get_row_idx_to_logic(self,state,row):
        row_idx = self.get_row_at_state_order(state)
        return row_idx.row_id.tolist().index(row)        

    def get_col_logic_to_idx(self,state,col):
        col_logic = self.get_col_at_state_order(state)
        return col_logic.iloc[col].col_id
    
    def get_col_idx_to_logic(self,state,col):
        col_idx = self.get_col_at_state_order(state)
        return col_idx.col_id.tolist().index(col) 

    def get_values_at_state(self,state):
        return pd.read_sql_query("""
        select ({state}-(select max(state_id) from state s))*-1 as state
        ,{state} as state_id,a.content_id,a.prev_content_id,c.value_text,d.row_id,d.col_id 
        from content a
        NATURAL JOIN value c
        NATURAL JOIN cell d
        where a.state_id<={state}
        and a.content_id not in
        (
        select a.prev_content_id from content a
        where a.state_id<={state}
        )
        """.format(state=state), self.conn)

    def get_snapshot_at_state(self,state):
        row_order = self.get_row_at_state_order(state)
        col_order = self.get_col_at_state_order(state)
        #print(col_order)
        values = self.get_values_at_state(state)

        max_row_id = row_order.row_id.max()
        max_col_id = col_order.col_id.max()
        new_arr = np.empty((max_row_id+1,max_col_id+1),dtype=object)
        #print(new_arr.shape)
        for x in values.to_records():
            try:
                new_arr[x.row_id,x.col_id] = x.value_text
            except:
                continue
        
        col_sort = sorted(col_order[["level","col_id","col_name"]].values.tolist(),key=lambda x:x[0])
        row_sort = sorted(row_order[["level","row_id"]].values.tolist(),key=lambda x:x[0])
        
        #print(col_names,row_names)
        snapshot_pd = pd.DataFrame(new_arr)
        #snapshot_pd.columns = col_names
        snapshot_pd = snapshot_pd.iloc[[x[1] for x in row_sort],[x[1] for x in col_sort]]
        snapshot_pd.columns = [x[2] for x in col_sort]
        return snapshot_pd

    def get_cell_history(self,row,col,is_id=True):
        # if it's a logical id, get the id
        if not is_id:
            col_id = self.get_col_logic_to_idx(state,col)
            row_id = self.get_row_logic_to_idx(state,row)
        else:
            col_id = col
            row_id = row
        
        return pd.read_sql_query("""
        select * from value_at_state a  where row_id={} and col_id={} order by state
        """.format(row_id,col_id), self.conn)

    def get_cell_lineage(self,row,col,is_id=True):
        pass



In [4]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [5]:
import os

db_files = []
for x in os.listdir():
    if x.endswith(".db"):
        db_files.append(x)

global orpe,num_state

@interact
def interactive_form(file=db_files):
    global orpe,num_state
    orpe = ProvenanceExplorer(file)
    num_state = orpe.get_number_of_state().num_state.values[0]
    


interactive(children=(Dropdown(description='file', options=('airbnb_demo.db', 'ipaw_2021_demo.db'), value='air…

# Changes at state

In [7]:
@interact
def interactive_form(start_state=range(0,num_state),end_state=range(0,num_state+2)):
    #return df.loc[df[column] > x]
    return orpe.get_changes_each_state(range(start_state,end_state+1))
#orpe.get_changes_each_state(2)

interactive(children=(Dropdown(description='start_state', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1…

# Column Schema changes at state

In [8]:
@interact
def interactive_form(start_state=range(0,num_state),end_state=range(0,num_state+2)):
    #return df.loc[df[column] > x]
    return orpe.get_col_at_state_order(range(start_state,end_state+1))

#orpe.get_col_at_state_order(range(2,5))

interactive(children=(Dropdown(description='start_state', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1…

# Row order at state

In [9]:
@interact
def interactive_form(state_id=range(0,num_state+2)):
    #return df.loc[df[column] > x]
    return orpe.get_row_at_state_order(state_id)

#orpe.get_row_at_state_order(5) 

interactive(children=(Dropdown(description='state_id', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

# Cell History

In [10]:
@interact
def interactive_form(col="0",row="3"):
    #return df.loc[df[column] > x]
    return orpe.get_cell_history(int(row),int(col))


interactive(children=(Text(value='0', description='col'), Text(value='3', description='row'), Output()), _dom_…

# Snapshot at state

In [12]:
@interact
def interactive_form(state=range(0,num_state+2)):
    #return df.loc[df[column] > x]
    return orpe.get_snapshot_at_state(state)


interactive(children=(Dropdown(description='state', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,…

In [177]:
orpe.get_col_at_state_order(1)

,state,col_id,col_name,prev_col_id,level
0,1,0,Title,-1,0
1,1,1,Author,0,1
2,1,2,Date,1,2


In [154]:
@interact
def interactive_form(x=range(0,num_state),y=range(0,num_state)):
    #return df.loc[df[column] > x]
    return orpe.get_column_at_state(range(x,y+1))


interactive(children=(Dropdown(description='x', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,…

In [108]:
tt = orpe.get_snapshot_at_state(0)

In [109]:
tt

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,30041063,(Quiet & Affordable Private BDR near UOC),225773707,LaShun test,None,Woodlawn,41.77712429,-87.59471115,Private room,25,1,0,None,None,1,173
1,29960994,(Feel cozy in chicago),225262670,Karla,None,Edgewater,41.98536071,-87.66898893,Private room,70,2,0,None,None,1,179
2,29946929,(Cozy 2 Bedroom Apartment in the Heart of Chic...,225157895,Frankie,None,Lower West Side,41.85198185,-87.67767597,Entire home/apt,79,1,0,None,None,1,62
3,29940545,(Beautiful 7Br 6200 sf Brick & Steel Custom Ma...,225106605,Neda,None,Bridgeport,41.84452795,-87.6558587,Entire home/apt,300,2,0,None,None,1,355
4,29936021,"(Convenient, Beautiful, Spacious 2Br/2Ba Condo)",225076770,J,None,Mckinley Park,41.8296729,-87.67826309,Entire home/apt,70,1,0,None,None,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7587,10610,(3 Comforts of Cooperative Living),2140,Lois And Ed,None,hyde park,41.79708495,-87.59194894,Private room,35,2,31,2018-07-29T00:00:00Z,0.63,5,179
7588,3907071,(2 A Walk in Hyde Park),2140,Lois And Ed,None,hyde park,41.79554928,-87.59174205,Private room,37,2,45,2018-09-28T00:00:00Z,0.91,5,232
7589,3983057,(4 Cooperative Living in Hyde Park),2140,Lois And Ed,None,hyde park,41.79507237,-87.59332266,Private room,40,2,38,2018-09-16T00:00:00Z,0.88,5,333
7590,3984079,(5 Sabbatical in Hyde Park),2140,Lois And Ed,None,hyde park,41.79718187,-87.59378733,Entire home/apt,150,2,10,2018-07-23T00:00:00Z,0.31,5,55


In [14]:
orpe.get_cell_history(3,2)

,state,state_id,content_id,prev_content_id,value_text,row_id,col_id
0,0,58,288897,83,225106605,3,2
1,1,57,288897,83,225106605,3,2
2,2,56,288897,83,225106605,3,2
3,3,55,288897,83,225106605,3,2
4,4,54,288897,83,225106605,3,2
5,5,53,288897,83,225106605,3,2
6,6,52,288897,83,225106605,3,2
7,7,51,288897,83,225106605,3,2
8,8,50,288897,83,225106605,3,2
9,9,49,83,-1,225106605,3,2


In [4]:
orpe.get_column_at_state(1)

,state,state_id,col_schema_id,col_id,col_name,prev_col_id,prev_col_schema_id
0,1,57,0,0,id,-1,-1
1,1,57,1,1,name,0,-1
2,1,57,2,22,name_grel,1,-1
3,1,57,3,23,name_grel_star,22,-1
4,1,57,9,16,neighbourhood_case,5,-1
5,1,57,10,17,neighbourhood_loop,16,-1
6,1,57,11,26,test_combine,17,-1
7,1,57,14,7,longitude,6,-1
8,1,57,15,8,room_type,7,-1
9,1,57,16,9,price,8,-1


In [6]:
orpe.get_col_at_state_order(0)

,state,col_id,col_name,prev_col_id,level
0,0,0,id,-1,0
1,0,1,name,0,1
2,0,22,name_grel,1,2
3,0,2,host_id,1,2
4,0,23,name_grel_star,22,3
5,0,3,host_name,2,3
6,0,4,neighbourhood_group,3,4
7,0,5,neighbourhood,4,5
8,0,16,neighbourhood_case,5,6
9,0,6,latitude,5,6


In [6]:
orpe.get_col_idx_to_logic(8,1)

1

In [7]:
orpe.get_col_logic_to_idx(8,2)

22

In [3]:
xx = orpe.get_row_at_state(1)
xx

In [13]:
orpe.get_row_at_state_order(0)

,state,state_id,row_id,prev_row_id,level
0,58,0,0,-1,0
1,58,0,1,0,1
2,58,0,2,1,2
3,58,0,3,2,3
4,58,0,4,3,4
...,...,...,...,...,...
7587,58,0,7587,7586,7587
7588,58,0,7588,7587,7588
7589,58,0,7589,7588,7589
7590,58,0,7590,7589,7590


In [175]:
orpe.get_row_logic_to_idx(4,2)

3

In [176]:
orpe.get_row_idx_to_logic(4,3)

2

In [177]:
orpe.get_values_at_state(0)

,state,state_id,content_id,prev_content_id,value_text,row_id,col_id
0,0,9,0,-1,Against Method,0,0
1,0,9,1,-1,"Feyerabend, P.",0,1
2,0,9,3,-1,Feyerabend,0,3
3,0,9,4,-1,None,0,4
4,0,9,5,-1,Changing Order,1,0
5,0,9,6,-1,"Collins, H.M.",1,1
6,0,9,8,-1,Collins,1,3
7,0,9,9,-1,None,1,4
8,0,9,10,-1,Exceeding Our Grasp,2,0
9,0,9,13,-1,Stanford,2,3


In [164]:
xx.row_pos_id.tolist().index(4)

2

In [22]:
list(orpe.cursor.execute("select * from col_each_state where state=8"))
names = list(map(lambda x: x[0], orpe.cursor.description))
names

['state',
 'state_id',
 'col_schema_id',
 'col_id',
 'col_name',
 'prev_col_id',
 'prev_col_schema_id']

In [21]:
xx = orpe.cursor.execute("select * from col_each_state where state=8")

['state',
 'state_id',
 'col_schema_id',
 'col_id',
 'col_name',
 'prev_col_id',
 'prev_col_schema_id']

In [16]:
orpe.get_column_at_state(5)

[(5, 4, 0, 0, 'Title', -1, -1),
 (5, 4, 8, 2, 'Date', 1, 1),
 (5, 4, 9, 1, 'Author', 0, 2)]